In [1]:
import sys
sys.path.append("../scripts")

In [4]:
## General
import pandas as pd
import numpy as np

## Data cleaning/setup
from clean_data import load_and_clean_data
from sklearn.model_selection import train_test_split
from sklearn.utils import resample

## Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

## Tuning
from sklearn.model_selection import GridSearchCV

## Evaluation metrics
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

## Other
pd.set_option('display.max_rows', 400)

In [3]:
def upsample(X, y):
    idx_class_1 = np.where(y == 1)[0]
    class_1 = X.iloc[idx_class_1]
    n_resample = len(X) - len(idx_class_1)
    X_upsampled = resample(class_1, n_samples = n_resample, random_state = 42)
    y_upsampled = y.loc[X_upsampled.index]
    X_up = pd.concat([X_upsampled, X.iloc[np.where(y == 0)[0]]])
    y_up = pd.concat([y_upsampled, y.iloc[np.where(y == 0)[0]]])
    
    return X_up, y_up

In [5]:
def run_model(X_train, y_train, X_val, y_val, model = DecisionTreeClassifier()):
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:,1]
    
    fpr, tpr, thresholds = roc_curve(y_val, y_pred)
    roc_auc = auc(fpr, tpr)
    
    return model, y_pred, roc_auc, fpr, tpr

## Data

#### Pull and split data

In [6]:
X_train, X_test, y_train, y_test = load_and_clean_data()

/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (10,17,19,23,25,27,28,29,30,32,33,36,37,38,39,40,41,42,43,50,51,83,86,89,92,94,95,97,100,103,106,110,113,116,121,124,127,130,133,136,188,190,201,212,231,232,242,244,247,249,256,262,267,273,279,280,281,282,283,284,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,305,318,320,333,334) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:163

In [7]:
selected_features = pd.read_csv('feature_selection_final_columns.csv', names=["feats"], skiprows=1)

In [8]:
selected_features = selected_features.feats.values

X_train = X_train[selected_features]

## Logistic Regression

In [19]:
X_train_up, y_train_up = upsample(X_train, y_train)

In [20]:
lr_model = LogisticRegression(solver='liblinear')
lr_params = {'penalty': ['none', 'l2', 'l1', 'elasticnet'],
            'C': [.01, 0.1, 1, 10],
            'max_iter': [100, 200, 300]}
lr_gs = GridSearchCV(lr_model, lr_params, scoring='roc_auc', n_jobs=-1, verbose=3)

In [ ]:
lr_gs.fit(X_train_up, y_train_up)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/kpant/.pyenv/versions/3.9.1/envs/cs/lib/python3.9/site-packages/sklearn/svm/_base.py:1199: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
lr_params_best = lr_gs.best_params_
lr_score_best = lr_gs.best_score_
print(f'Best score: {lr_score_best}, Best params: {lr_params_best}')

## Decision Tree

In [15]:
dt_model = DecisionTreeClassifier()
dt_params = {'max_depth':[None, 2, 3, 5, 10], 
             'min_samples_split':[2, 10, 100, 1000, 10000, 50000], 
             'min_samples_leaf':[2, 10, 100, 1000, 10000, 50000]}
dt_gs = GridSearchCV(dt_model, dt_params, scoring='roc_auc', n_jobs=-1, verbose=3)

In [16]:
dt_gs.fit(X_train, y_train)

Fitting 5 folds for each of 180 candidates, totalling 900 fits


GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 2, 3, 5, 10],
                         'min_samples_leaf': [2, 10, 100, 1000, 10000, 50000],
                         'min_samples_split': [2, 10, 100, 1000, 10000, 50000]},
             scoring='roc_auc', verbose=3)

[CV 1/5] END max_depth=None, min_samples_leaf=2, min_samples_split=2;, score=0.736 total time=  15.4s
[CV 1/5] END max_depth=None, min_samples_leaf=2, min_samples_split=100;, score=0.796 total time=  13.2s
[CV 3/5] END max_depth=None, min_samples_leaf=2, min_samples_split=1000;, score=0.814 total time=  12.7s
[CV 5/5] END max_depth=None, min_samples_leaf=2, min_samples_split=50000;, score=0.776 total time=   6.0s
[CV 2/5] END max_depth=None, min_samples_leaf=10, min_samples_split=10;, score=0.763 total time=  12.7s
[CV 5/5] END max_depth=None, min_samples_leaf=10, min_samples_split=100;, score=0.784 total time=  11.3s
[CV 4/5] END max_depth=None, min_samples_leaf=10, min_samples_split=10000;, score=0.810 total time=   8.0s
[CV 2/5] END max_depth=None, min_samples_leaf=100, min_samples_split=2;, score=0.788 total time=   7.7s
[CV 5/5] END max_depth=None, min_samples_leaf=100, min_samples_split=10;, score=0.788 total time=   8.0s
[CV 3/5] END max_depth=None, min_samples_leaf=100, min_sam

In [18]:
dt_params_best = dt_gs.best_params_
dt_score_best = dt_gs.best_score_
print(f'Best score: {dt_score_best}, Best params: {dt_params_best}')

Best score: 0.8164791492254946, Best params: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 1000}


## Random Forest

In [35]:
rf_model = RandomForestClassifier()
rf_params = {
    'max_depth':[None, 5, 10, 50, 100], 
    'min_samples_split':[2, 10, 100, 1000, 10000, 50000], 
    'min_samples_leaf':[2, 10, 100, 1000, 10000, 50000],
    'n_estimators':[10, 50, 100, 200]
    }
rf_gs = GridSearchCV(rf_model, rf_params, scoring='roc_auc', n_jobs=-1, verbose=3)

In [36]:
rf_gs.fit(X_train, y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 42.2min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed: 55.5min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 60.6min
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed: 71.0min
[Parallel(n_jobs=-1)]: Done 1552 tasks      | elapsed: 82.5min
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed: 104.2min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 153.9min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 200.6min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed: 214.0min finished


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 5, 10, 50, 100],
                         'min_samples_leaf': [2, 10, 100, 1000, 10000, 50000],
                         'min_samples_split': [2, 10, 100, 1000, 10000, 50000],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='roc_auc', verbose=3)

In [37]:
rf_params_best = rf_gs.best_params_
rf_score_best = rf_gs.best_score_
print(f'Best score: {rf_score_best}, Best params: {rf_params_best}')

Best score: 0.8304663288784431, Best params: {'max_depth': 50, 'min_samples_leaf': 10, 'min_samples_split': 10, 'n_estimators': 200}


In [38]:
rf_df = pd.DataFrame(rf_gs.cv_results_['params'])
rf_df['auc'] = rf_gs.cv_results_['mean_test_score']

In [39]:
rf_df

,max_depth,min_samples_leaf,min_samples_split,n_estimators,auc
0,NaN,2,2,10,0.796901
1,NaN,2,2,50,0.819508
2,NaN,2,2,100,0.823513
3,NaN,2,2,200,0.827595
4,NaN,2,10,10,0.804377
...,...,...,...,...,...
715,100.0,50000,10000,200,0.555695
716,100.0,50000,50000,10,0.521904
717,100.0,50000,50000,50,0.554085
718,100.0,50000,50000,100,0.555374


In [40]:
rf_df.to_csv('rf_hyperparams.csv')

## XGBoost

In [17]:
xgb_model = XGBClassifier()
xgb_params = {'max_depth': [2, 6, 10, 20],
              'eta': [0.1, 0.2, 0.5, 0.75, 1], 
              'eval_metric':['auc']}
xgb_gs = GridSearchCV(xgb_model, xgb_params, scoring='roc_auc', n_jobs=-1, verbose=3)

In [18]:
xgb_gs.fit(X_train, y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 63.0min finished
/Users/saraokun/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
            

In [19]:
xgb_params_best = xgb_gs.best_params_
xgb_score_best = xgb_gs.best_score_
print(f'Best score: {xgb_score_best}, Best params: {xgb_params_best}')

Best score: 0.8451799972636577, Best params: {'eta': 0.2, 'eval_metric': 'auc', 'max_depth': 6}


In [22]:
xgb_gs.cv_results_

{'mean_fit_time': array([ 59.59231124, 182.66312904, 306.18019161, 776.27782845,
         50.02427506, 155.41358767, 297.53343463, 704.09414616,
         49.97850485, 146.63416457, 263.82119246, 690.34791198,
         43.64460769, 148.39301758, 303.65110302, 697.15628376,
         52.16836004, 162.90340872, 284.6257483 , 446.82943039]),
 'std_fit_time': array([ 0.10767439,  1.83819943, 12.38322255,  5.30562937,  0.59625533,
         3.75554809,  8.57213632, 13.66887472,  0.80048545,  8.49763275,
         2.82595408,  5.14764714,  2.01982458,  2.00844831,  4.50952665,
         4.29625628,  4.25285025,  3.33420535, 11.85665871, 44.00706359]),
 'mean_score_time': array([0.28325348, 0.68361588, 0.62394032, 1.03745027, 0.35537386,
        0.47546816, 0.64677844, 0.88735757, 0.41875134, 0.41612444,
        0.44577174, 0.92092438, 0.2686316 , 0.40999713, 0.65550442,
        0.81555705, 0.42051792, 0.55980196, 0.56535234, 0.4974936 ]),
 'std_score_time': array([0.03146172, 0.05991953, 0.052383

In [30]:
xgb_df = pd.DataFrame(xgb_gs.cv_results_['params'])
xgb_df['auc'] = xgb_gs.cv_results_['mean_test_score']

In [31]:
xgb_df

,eta,eval_metric,max_depth,auc
0,0.10,auc,2,0.809783
1,0.10,auc,6,0.841515
2,0.10,auc,10,0.844897
3,0.10,auc,20,0.835604
4,0.20,auc,2,0.828277
5,0.20,auc,6,0.845180
6,0.20,auc,10,0.840671
7,0.20,auc,20,0.814390
8,0.50,auc,2,0.840481
9,0.50,auc,6,0.834285


In [32]:
xgb_df.to_csv('xgb_hyperparams.csv')